In [1]:
#Sentiment Analysis is the technique to understand the sentiment behind the text. In this assignment sentiment analysis has been done on the reviews
#given by the students on movies or products. The data was collected through a google form. 

#Form Link - https://docs.google.com/forms/d/1Xnu_T5FRgxBFBJAucqqsZKTctgB71grbo3nFy3XKwqs/edit
#Response Link - https://docs.google.com/spreadsheets/d/1LnKBY_wucqR1MjdmnWLaZV7yl8OwwmypuYES8KsYa-w/edit?gid=1848984344#gid=1848984344


In [167]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import nltk
import re
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from xgboost import XGBRFClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

[nltk_data] Downloading package stopwords to C:\Users\Abhinav
[nltk_data]     Khandelwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [168]:
df = pd.read_csv(r"C:\Users\Abhinav Khandelwal\Desktop\Machine Learning\Sentiment Analysis using Machine Learning\Sentiment Data  - Form Responses 2.csv", usecols = ["Mention your comment here it can be related to the movie or any product.",
                                                                                                                           "Sentiment of the above comment"])

In [169]:
df.columns = ["Reviews","Sentiments"]

In [170]:
df.head(5)

,Reviews,Sentiments
0,Recently I have purchaced a shoe from decathel...,Positive
1,L2: Empuraan \nits a good movie to watch .,Positive
2,Absolutely loved it! The storyline kept me hoo...,Positive
3,The acting was phenomenal and the visuals were...,Positive
4,Great user experience – smooth and intuitive d...,Positive


In [171]:
df["Sentiments"].value_counts()

Sentiments
Positive    74
Negative    68
Neutral     56
Name: count, dtype: int64

# Data Cleaning and Embedding 

In [172]:
def cleaning(text):
    clean = text.lower()
    reg = re.sub(r"[!\n@#$%^&*()-_+=[\]{}|;:,.<>?/\'`~\.]"," ",clean)
    return reg

In [173]:
df["Reviews"] = df["Reviews"].apply(cleaning)

In [174]:
def remove_stopwords(text):
    txt = []
    for word in text.split():
        if word not in stopwords.words("english"):
            txt.append(word)
        
            
    e = txt[:]
    txt.clear()
    return e
    

In [175]:
df["Reviews"] = df["Reviews"].apply(remove_stopwords)

In [176]:
def join(text):
    return " ".join(text)

In [177]:
df["Reviews"] = df["Reviews"].apply(join)

In [178]:
x_train, x_test, y_train, y_test = train_test_split(df["Reviews"], df["Sentiments"], test_size=0.3, random_state=42)

In [179]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [180]:
cv = CountVectorizer(ngram_range=(1,4))

In [181]:
x_train_vector = cv.fit_transform(x_train).toarray()
x_test_vector = cv.transform(x_test).toarray()

In [182]:
pd.DataFrame(data = x_train_vector, columns = cv.get_feature_names_out())

,absolutely,absolutely loved,absolutely loved storyline,absolutely loved storyline kept,acting,acting especially,acting especially lead,acting especially lead actor,acting mediocre,acting mediocre story,...,works exactly described minor,worth,worth money,would,would definitely,would definitely buy,would definitely buy brand,would suggest,would suggest anyone,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [183]:
#Over Sampling due to Imbalance Data
over = SMOTE(k_neighbors=5)
x_train_o, y_train_o = over.fit_resample(x_train_vector, y_train)

In [184]:
#Applying PCA Feature Decomposition
pca = PCA(n_components=90,n_oversamples=50)
x_train_new = pca.fit_transform(x_train_o)
x_test_new = pca.transform(x_test_vector)

In [185]:
clf1 = DecisionTreeClassifier()
clf2 = SVC()
clf3 = RandomForestClassifier()
clf4 = GradientBoostingClassifier()
clf5 = XGBRFClassifier()

names = ["Decision Tree", "Support Vector", "Random Forest", "Gradient","XGBoost"]

for clf, name in zip([clf1,clf2,clf3,clf4,clf5],names):
    model = clf.fit(x_train_o,y_train_o)
    y_pred = model.predict(x_test_vector)
    print(f"Accuracy Score = {accuracy_score(y_test, y_pred)}, -- {name}")

Accuracy Score = 0.4166666666666667, -- Decision Tree
Accuracy Score = 0.4, -- Support Vector
Accuracy Score = 0.4166666666666667, -- Random Forest
Accuracy Score = 0.45, -- Gradient
Accuracy Score = 0.43333333333333335, -- XGBoost


# Optuna Hyper Parameter Tuning

In [78]:
def objective(trial):
    n_estimators= trial.suggest_int("n_estimators", 62, 70)
    max_depth = trial.suggest_int("max_depth", 7, 12)
    learning_rate =  trial.suggest_float("learning_rate", 0.005, 0.008, log=True)
    model = XGBRFClassifier(n_estimators =  n_estimators,max_depth =  max_depth, learning_rate=learning_rate)
    model.fit(x_train_o, y_train_o)
    y_pred = model.predict(x_test_vector)
    score = accuracy_score(y_test, y_pred)
    return score

In [79]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-06-07 08:51:52,155] A new study created in memory with name: no-name-074f0f75-98a5-4763-9baa-d9ba92f05c50
[I 2025-06-07 08:51:53,802] Trial 0 finished with value: 0.5333333333333333 and parameters: {'n_estimators': 69, 'max_depth': 10, 'learning_rate': 0.006414796407310343}. Best is trial 0 with value: 0.5333333333333333.
[I 2025-06-07 08:51:55,480] Trial 1 finished with value: 0.5333333333333333 and parameters: {'n_estimators': 70, 'max_depth': 10, 'learning_rate': 0.007114383796294974}. Best is trial 0 with value: 0.5333333333333333.
[I 2025-06-07 08:51:56,678] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 68, 'max_depth': 7, 'learning_rate': 0.005066909500674922}. Best is trial 0 with value: 0.5333333333333333.
[I 2025-06-07 08:51:58,452] Trial 3 finished with value: 0.5166666666666667 and parameters: {'n_estimators': 70, 'max_depth': 11, 'learning_rate': 0.007728254679603975}. Best is trial 0 with value: 0.5333333333333333.
[I 2025-06-07 08:52:00,109] Tr

In [88]:
clf = XGBRFClassifier(n_estimators =  69, max_depth =  10, learning_rate = 0.006414796407310343)
clf.fit(x_train_o,y_train_o)
pred = clf.predict(x_test_vector)
accuracy_score(y_test,pred)

0.5333333333333333

# Taking new Input

In [186]:
clean = FunctionTransformer(cleaning)
remove_stop = FunctionTransformer(remove_stopwords)
join = FunctionTransformer(join)

In [189]:
new_review = "This movie is very good and we should watch it"

In [190]:
pip_clean = Pipeline(
    [
    ("clean",clean),
    ("stop",remove_stop),
    ("join",join),
    ]
)
f_text = pip_clean.transform(new_review)

cv_text = cv.transform([f_text]).toarray()
pr = clf.predict(cv_text)
if(pr == 0):
    print("Negative Sentiment")
if(pr == 1):
    print("Neutral Sentiment")
elif pr == 2:
    print("Positive Sentiment")

Positive Sentiment
